In [ ]:
# !pip install segmentation-models-pytorch
# !pip install pytorch_msssim
# !pip install lightning
# !pip install lightning[extra]
# !pip install tensorboard

In [1]:
import random
import numpy as np
import os

import torch
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import cv2

import zipfile


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [2]:
print(torch.cuda.is_available())  # True인지 확인
print(torch.cuda.device_count())  # 사용 가능한 GPU 개수 확인
print(torch.cuda.current_device())  # 현재 사용 중인 GPU 인덱스 확인
print(torch.cuda.get_device_name(0))  # 사용 중인 GPU 이름 확인

True
1
0
NVIDIA GeForce RTX 4070 Laptop GPU


In [3]:
CFG = {
    'EPOCHS':2,
    'LEARNING_RATE':3e-4,
    # 'BATCH_SIZE':16,
    'BATCH_SIZE':64,
    'SEED':42
}

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [5]:
def get_input_image(image):
    # 이 함수는 손상된 이미지로부터 마스크를 생성하여 반환합니다.
    # 예시: 특정 조건에 따라 임의로 마스크 생성 (여기서는 단순히 모든 픽셀을 마스크로 가정)
    
    # 손상된 이미지의 그레이스케일 버전과 마스크 생성 (단순한 예제)
    image_gray = image.convert("L")
    mask = np.array(image_gray) > 128  # 임의의 임계값 기준으로 마스크 생성
    mask = Image.fromarray(mask.astype(np.uint8) * 255)  # 마스크 이미지를 PIL 형식으로 변환
    
    return {
        'image_gray_masked': image_gray,
        'mask': transforms.ToTensor()(mask)  # 마스크를 텐서로 변환하여 사용
    }

In [6]:
class CustomDataset(Dataset):
    def __init__(self, damage_dir, origin_dir, transform=None, use_masks=False):
        self.damage_dir = damage_dir
        self.origin_dir = origin_dir
        self.transform = transform
        self.use_masks = use_masks  # 마스크 사용 여부
        self.damage_files = sorted(os.listdir(damage_dir))
        self.origin_files = sorted(os.listdir(origin_dir))

    def __len__(self):
        return len(self.damage_files)

    def __getitem__(self, idx):
        damage_img_name = self.damage_files[idx]
        origin_img_name = self.origin_files[idx]

        damage_img_path = os.path.join(self.damage_dir, damage_img_name)
        origin_img_path = os.path.join(self.origin_dir, origin_img_name)

        damage_img = Image.open(damage_img_path).convert("RGB")
        origin_img = Image.open(origin_img_path).convert("RGB")

        if self.use_masks:
            input_data = get_input_image(damage_img)  # 손상된 이미지에서 마스크 생성
            mask = input_data['mask']
            damage_img = input_data['image_gray_masked']
        else:
            mask = torch.zeros((1, damage_img.size[1], damage_img.size[0]))  # 빈 마스크 생성

        if self.transform:
            damage_img = self.transform(damage_img)
            origin_img = self.transform(origin_img)

        return {'A': damage_img, 'B': origin_img, 'mask': mask}

In [7]:
from segmentation_models_pytorch import UnetPlusPlus
import torch.nn.functional as F
import lightning as L
from skimage.metrics import structural_similarity as ssim
import cv2
import numpy as np
import torch

def get_histogram_similarity(true, pred, color_space=cv2.COLOR_RGB2HSV):
    # PyTorch 텐서를 NumPy 배열로 변환하고 차원을 조정
    true_np = true.permute(1, 2, 0).cpu().numpy().astype(np.uint8)
    pred_np = pred.permute(1, 2, 0).cpu().numpy().astype(np.uint8)
    
    # 히스토그램 유사도 계산
    true_hsv = cv2.cvtColor(true_np, color_space)
    pred_hsv = cv2.cvtColor(pred_np, color_space)
    
    # 히스토그램 계산 및 비교
    hist_true = cv2.calcHist([true_hsv], [0], None, [180], [0, 180])
    hist_pred = cv2.calcHist([pred_hsv], [0], None, [180], [0, 180])
    
    hist_true = cv2.normalize(hist_true, hist_true).flatten()
    hist_pred = cv2.normalize(hist_pred, hist_pred).flatten()
    
    similarity = cv2.compareHist(hist_true, hist_pred, cv2.HISTCMP_CORREL)
    
    return similarity

def get_masked_ssim_score(true, pred, mask):
    # PyTorch 텐서를 NumPy 배열로 변환하고, 차원을 조정
    true_np = true.permute(1, 2, 0).cpu().numpy()
    pred_np = pred.permute(1, 2, 0).cpu().numpy()
    mask_np = mask.permute(1, 2, 0).cpu().numpy()

    # 마스크가 적용된 부분만 추출
    true_masked = true_np[mask_np > 0]
    pred_masked = pred_np[mask_np > 0]

    if true_masked.size == 0 or pred_masked.size == 0:
        return 0  # 마스크된 부분이 없으면 SSIM을 0으로 처리

    # SSIM 계산
    ssim_value = ssim(
        true_masked, pred_masked, data_range=pred_masked.max() - pred_masked.min(), channel_axis=-1
    )
    return ssim_value

def ssim_score(true, pred):
    true_np = true.permute(0, 2, 3, 1).cpu().numpy()
    pred_np = pred.permute(0, 2, 3, 1).cpu().numpy()
    scores = [
        ssim(t, p, channel_axis=-1, data_range=p.max() - p.min())
        for t, p in zip(true_np, pred_np)
    ]
    return np.mean(scores)

def get_ssim_score(true, pred):
    # PyTorch 텐서를 NumPy 배열로 변환하고, 차원을 조정
    true_np = true.permute(0, 2, 3, 1).cpu().numpy()
    pred_np = pred.permute(0, 2, 3, 1).cpu().numpy()
    scores = [
        ssim(t, p, channel_axis=-1, data_range=p.max() - p.min())
        for t, p in zip(true_np, pred_np)
    ]
    return np.mean(scores)

# LitIRModel 클래스 정의
class LitIRModel(L.LightningModule):
    def __init__(self, model_1, model_2, image_mean=0.5, image_std=0.5):
        super().__init__()
        self.model_1 = model_1
        self.model_2 = model_2
        self.image_mean = image_mean
        self.image_std = image_std

    def forward(self, images_gray_masked):
        # 첫 번째 모델: Gray Mask Restoration
        images_gray_restored = self.model_1(images_gray_masked) + images_gray_masked
        # 두 번째 모델: Gray → Color
        images_restored = self.model_2(images_gray_restored)
        return images_gray_restored, images_restored

    def unnormalize(self, output, round=False):
        # 출력값을 unnormalize
        image_restored = ((output * self.image_std + self.image_mean) * 255).clamp(0, 255)
        if round:
            image_restored = torch.round(image_restored)
        return image_restored

    def configure_optimizers(self):
        # AdamW Optimizer
        opt = torch.optim.AdamW(self.parameters(), lr=1e-5)
        return opt

    def training_step(self, batch, batch_idx):
        # GPU 이동 명시적으로 지정
        images_gray_masked = batch['images_gray_masked'].to(self.device)
        images_gt = batch['images_gt'].to(self.device)
        
        # Forward pass
        images_gray_restored, images_restored = self(images_gray_masked)
        
        # Forward pass
        images_gray_restored, images_restored = self(images_gray_masked)

        # Loss 계산
        loss_pixel_gray = (
            F.l1_loss(images_gray, images_gray_restored, reduction='mean') * 0.5 +
            F.mse_loss(images_gray, images_gray_restored, reduction='mean') * 0.5
        )
        loss_pixel = (
            F.l1_loss(images_gt, images_restored, reduction='mean') * 0.5 +
            F.mse_loss(images_gt, images_restored, reduction='mean') * 0.5
        )
        loss = loss_pixel_gray * 0.5 + loss_pixel * 0.5

        print(f"Batch {batch_idx}, Loss: {loss.item()}")
        # Logging (매 배치마다 메트릭을 로깅하지 않고, 에폭 종료 시 로깅)
        self.log("train_loss", loss, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # 10번째 배치마다 메트릭 계산
        if batch_idx % 10 == 0:
            masks, images_gray_masked, images_gt = (
                batch['masks'], batch['images_gray_masked'], batch['images_gt']
            )
            images_gray_restored, images_restored = self(images_gray_masked)

            # Unnormalize for evaluation
            images_gt, images_restored = self.unnormalize(images_gt, round=True), self.unnormalize(images_restored, round=True)
            masks_np = masks.detach().cpu().numpy()
            images_gt_np = images_gt.detach().cpu().permute(0, 2, 3, 1).float().numpy().astype(np.uint8)
            images_restored_np = images_restored.detach().cpu().permute(0, 2, 3, 1).float().numpy().astype(np.uint8)

            # Metric 계산
            total_ssim_score = 0
            masked_ssim_score = 0
            hist_sim_score = 0
            for image_gt_np, image_restored_np, mask_np in zip(images_gt_np, images_restored_np, masks_np):
                total_ssim_score += get_ssim_score(image_gt_np, image_restored_np) / len(images_gt)
                masked_ssim_score += get_masked_ssim_score(image_gt_np, image_restored_np, mask_np) / len(images_gt)
                hist_sim_score += get_histogram_similarity(image_gt_np, image_restored_np, cv2.COLOR_RGB2HSV) / len(images_gt)

            # 최종 점수
            score = total_ssim_score * 0.2 + masked_ssim_score * 0.4 + hist_sim_score * 0.4

            # Logging
            self.log(f"val_score", score, on_step=False, on_epoch=True)
            self.log(f"val_total_ssim_score", total_ssim_score, on_step=False, on_epoch=True)
            self.log(f"val_masked_ssim_score", masked_ssim_score, on_step=False, on_epoch=True)
            self.log(f"val_hist_sim_score", hist_sim_score, on_step=False, on_epoch=True)

        return None  # loss 반환 필요 없음

# 모델 초기화
# 첫 번째 모델: Gray Mask Restoration
model_1 = UnetPlusPlus(
    encoder_name="efficientnet-b4",
    encoder_weights="imagenet",
    in_channels=1,
    classes=1
)

# 두 번째 모델: Gray → Color
model_2 = UnetPlusPlus(
    encoder_name="efficientnet-b4",
    encoder_weights="imagenet",
    in_channels=1,
    classes=3
)

# LitIRModel 초기화
lit_ir_model = LitIRModel(model_1=model_1, model_2=model_2)


c:\Users\zqrc0\anaconda3\envs\new_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from torch.utils.data import random_split, DataLoader
import torchvision.transforms as transforms
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
import lightning as L

# 경로 설정
origin_dir = 'data/train_gt'  # 원본 이미지 폴더 경로
damage_dir = 'data/train_input'  # 손상된 이미지 폴더 경로
test_dir = 'data/test_input'     # test 이미지 폴더 경로

# 데이터 전처리 설정
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# 전체 데이터셋 생성
dataset = CustomDataset(damage_dir=damage_dir, origin_dir=origin_dir, transform=transform)

# 데이터셋을 학습과 검증으로 나누기 (예: 80% 학습, 20% 검증)
validation_ratio = 0.2
train_size = int((1 - validation_ratio) * len(dataset))
val_size = len(dataset) - train_size
training_dataset, validation_dataset = random_split(dataset, [train_size, val_size])

torch.set_float32_matmul_precision('high')

class CollateFn:
    def __init__(self, mode='train'):
        self.mode = mode

    def __call__(self, batch):
        # 'batch'는 각 데이터 샘플로 이루어진 리스트이며, 각 샘플은 {'A': tensor, 'B': tensor, 'mask': tensor or None} 형태
        A = torch.stack([item['A'] for item in batch])
        B = torch.stack([item['B'] for item in batch])
        
        if batch[0]['mask'] is not None:
            masks = torch.stack([item['mask'] for item in batch])
        else:
            masks = None
        
        if self.mode == 'train':
            return {'A': A, 'B': B, 'mask': masks}
        elif self.mode == 'valid':
            return {'A': A, 'B': B, 'mask': masks}

# DataLoader 설정 시 사용할 수 있도록 CollateFn 클래스 추가
train_collate_fn = CollateFn(mode='train')
validation_collate_fn = CollateFn(mode='valid')

# 학습 및 검증 DataLoader 설정
train_dataloader = DataLoader(
    training_dataset,
    batch_size=CFG['BATCH_SIZE'],
    shuffle=True,
    num_workers=8,
    pin_memory=True,  # 이 옵션 추가
    collate_fn=train_collate_fn
)

validation_dataloader = DataLoader(
    validation_dataset,
    batch_size=CFG['BATCH_SIZE'],
    shuffle=False,
    num_workers=8,
    collate_fn=validation_collate_fn
)

# DataLoader에서 배치 데이터를 GPU로 이동
for batch in train_dataloader:
    inputs = batch['A'].to(device)
    targets = batch['B'].to(device)



# 모델 저장 디렉토리 생성
model_save_dir = "./saved_models"
os.makedirs(model_save_dir, exist_ok=True)

# 3. Trainer 설정 및 학습 시작
trainer = L.Trainer(
    max_epochs=CFG['EPOCHS'],
    precision=16,
    accelerator='gpu',
    devices=1,
    log_every_n_steps=10,  # 매 10번째 스텝마다 로그를 남기도록 설정
    callbacks=[
        ModelCheckpoint(
            monitor='val_score',
            mode='max',
            save_top_k=1,
            dirpath=model_save_dir,
            filename='best_model-{epoch:02d}-{val_score:.4f}'
        ),
        EarlyStopping(monitor='val_score', mode='max', patience=3)
    ]
)

# 학습 시작
trainer.fit(lit_ir_model, train_dataloader, validation_dataloader)

In [ ]:
# import os
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader
# from IPython.display import FileLink
# import numpy as np
# from skimage.metrics import structural_similarity as ssim
# import cv2
# import torch.nn.functional as F
# from lightning.pytorch import Trainer

# class GeneratorModel(nn.Module):
#     def __init__(self):
#         super(GeneratorModel, self).__init__()
#         # 예시로 간단한 U-Net 아키텍처를 사용
#         self.encoder = nn.Sequential(
#             nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
#             nn.ReLU(True),
#             nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
#             nn.BatchNorm2d(128),
#             nn.ReLU(True)
#         )
#         self.decoder = nn.Sequential(
#             nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
#             nn.BatchNorm2d(64),
#             nn.ReLU(True),
#             nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
#             nn.Tanh()
#         )

#     def forward(self, x):
#         x = self.encoder(x)
#         x = self.decoder(x)
#         return x
    
# class PatchGANDiscriminator(nn.Module):
#     def __init__(self):
#         super(PatchGANDiscriminator, self).__init__()
#         self.model = nn.Sequential(
#             nn.Conv2d(6, 64, kernel_size=4, stride=2, padding=1),  # Real 이미지와 Fake 이미지가 concatenated 된 상태로 입력됨 (3+3=6 채널)
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
#             nn.BatchNorm2d(128),
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Conv2d(128, 1, kernel_size=4, stride=1, padding=1)
#         )

#     def forward(self, x, y):
#         # x는 손상된 이미지, y는 생성된 이미지 또는 원본 이미지
#         input = torch.cat([x, y], dim=1)  # 두 이미지를 채널 차원에서 concatenate
#         return self.model(input)

# # CUDA 성능 최적화를 위한 설정
# torch.backends.cudnn.benchmark = True  # 입력 크기가 일정하다면 성능을 최적화합니다.
# torch.backends.cudnn.enabled = True

# # 모델 저장 디렉토리 생성
# model_save_dir = "./saved_models"
# os.makedirs(model_save_dir, exist_ok=True)
# log_file = os.path.join(model_save_dir, "training_log.txt")

# # 초기값 설정
# best_loss = float("inf")
# best_score = float("-inf")
# lambda_pixel = 100  # 픽셀 손실 가중치

# # 손실 함수 정의
# criterion_GAN = nn.MSELoss()
# criterion_pixelwise = nn.L1Loss()

# # 히스토그램 유사도 계산 함수
# def histogram_similarity(true, pred):
#     true_np = true.permute(0, 2, 3, 1).cpu().numpy().astype(np.uint8)
#     pred_np = pred.permute(0, 2, 3, 1).cpu().numpy().astype(np.uint8)
#     similarities = []
#     for t, p in zip(true_np, pred_np):
#         true_hsv = cv2.cvtColor(t, cv2.COLOR_RGB2HSV)
#         pred_hsv = cv2.cvtColor(p, cv2.COLOR_RGB2HSV)
#         hist_true = cv2.calcHist([true_hsv], [0], None, [180], [0, 180])
#         hist_pred = cv2.calcHist([pred_hsv], [0], None, [180], [0, 180])
#         hist_true = cv2.normalize(hist_true, hist_true).flatten()
#         hist_pred = cv2.normalize(hist_pred, hist_pred).flatten()
#         similarities.append(cv2.compareHist(hist_true, hist_pred, cv2.HISTCMP_CORREL))
#     return np.mean(similarities)

# # SSIM 점수 계산 함수
# def ssim_score(true, pred):
#     true_np = true.permute(0, 2, 3, 1).cpu().numpy()
#     pred_np = pred.permute(0, 2, 3, 1).cpu().numpy()
#     scores = [ssim(t, p, channel_axis=-1, data_range=p.max() - p.min()) for t, p in zip(true_np, pred_np)]
#     return np.mean(scores)

# # Masked SSIM 점수 계산 함수 (데이터 크기 조정 오류 수정)
# def masked_ssim_score(true, pred, mask):
#     true_np = true.permute(0, 2, 3, 1).cpu().numpy()
#     pred_np = pred.permute(0, 2, 3, 1).cpu().numpy()
#     mask_np = mask.permute(0, 2, 3, 1).cpu().numpy()  # mask의 크기를 이미지와 동일하게 조정
#     scores = []
#     for t, p, m in zip(true_np, pred_np, mask_np):
#         if m.any():  # 마스크에 유효한 값이 있을 때만 계산
#             true_masked = t[m > 0]
#             pred_masked = p[m > 0]
#             if true_masked.size > 0 and pred_masked.size > 0:  # 유효한 픽셀이 존재할 때만 계산
#                 scores.append(
#                     ssim(
#                         true_masked, pred_masked, channel_axis=-1, 
#                         data_range=pred_masked.max() - pred_masked.min()
#                     )
#                 )
#     return np.mean(scores) if scores else 0.0

# # 모델 초기화 및 옵티마이저 설정
# def initialize_models_and_optimizers():
#     generator = GeneratorModel().to(device)  # GeneratorModel을 실제 모델로 변경
#     discriminator = PatchGANDiscriminator().to(device)
#     optimizer_G = optim.Adam(generator.parameters(), lr=CFG["LEARNING_RATE"])
#     optimizer_D = optim.Adam(discriminator.parameters(), lr=CFG["LEARNING_RATE"])
#     return generator, discriminator, optimizer_G, optimizer_D

# # 학습 단계 함수
# def train_step(generator, discriminator, optimizer_G, optimizer_D, batch):
#     real_A = batch['A'].to(device)  # 손상된 이미지
#     real_B = batch['B'].to(device)  # 원본 이미지
#     mask = batch.get('mask', None)  # 'mask'가 없을 경우 None으로 설정
#     if mask is not None and torch.any(mask):
#         mask = mask.to(device)
#     else:
#         mask = None

#     # Generator 훈련
#     optimizer_G.zero_grad()
#     fake_B = generator(real_A)

#     # 마스크된 부분만 손실 계산
#     if mask is not None:
#         fake_B_masked = fake_B * mask
#         real_B_masked = real_B * mask
#         loss_pixel = criterion_pixelwise(fake_B_masked, real_B_masked)
#     else:
#         loss_pixel = criterion_pixelwise(fake_B, real_B)
    
#     pred_fake = discriminator(fake_B, real_A)
#     loss_GAN = criterion_GAN(pred_fake, torch.ones_like(pred_fake).to(device))
#     loss_G = loss_GAN + lambda_pixel * loss_pixel
#     loss_G.backward()
#     optimizer_G.step()

#     # Discriminator 훈련
#     optimizer_D.zero_grad()
#     pred_real = discriminator(real_B, real_A)
#     loss_real = criterion_GAN(pred_real, torch.ones_like(pred_real).to(device))
#     pred_fake = discriminator(fake_B.detach(), real_A)
#     loss_fake = criterion_GAN(pred_fake, torch.zeros_like(pred_fake).to(device))
#     loss_D = 0.5 * (loss_real + loss_fake)
#     loss_D.backward()
#     optimizer_D.step()

#     return loss_G.item(), loss_D.item()

# # 검증 단계 함수
# def validate(generator, validation_dataloader):
#     generator.eval()
#     validation_loss, ssim_scores, masked_ssim_scores, hist_similarities = 0, [], [], []
#     with torch.no_grad():
#         for val_batch in validation_dataloader:
#             real_val_A = val_batch['A'].to(device)
#             real_val_B = val_batch['B'].to(device)
#             fake_val_B = generator(real_val_A)
    
#             # SSIM 점수 계산
#             ssim_value = ssim_score(real_val_B, fake_val_B)
#             ssim_scores.append(ssim_value)
    
#             # 마스크가 있을 경우에만 masked_ssim 계산
#             real_val_mask = val_batch.get('mask', None)
#             if real_val_mask is not None:
#                 real_val_mask = real_val_mask.to(device)
#                 masked_ssim_value = masked_ssim_score(real_val_B, fake_val_B, real_val_mask)
#                 masked_ssim_scores.append(masked_ssim_value)
    
#             # 히스토그램 유사도 계산
#             hist_value = histogram_similarity(real_val_B, fake_val_B)
#             hist_similarities.append(hist_value)

#             # 검증 손실 계산 (루프 내부에서 누적)
#             val_loss = criterion_pixelwise(fake_val_B, real_val_B).item()
#             validation_loss += val_loss
        
#     # 검증 루프 이후에 평균 손실 계산 (루프 외부)
#     validation_loss /= len(validation_dataloader)

#     # 평균값 계산
#     S = sum(ssim_scores) / len(ssim_scores)
#     M = sum(masked_ssim_scores) / len(masked_ssim_scores) if len(masked_ssim_scores) > 0 else 0
#     C = sum(hist_similarities) / len(hist_similarities)
#     score = (0.2 * S) + (0.4 * M) + (0.4 * C)

#     return validation_loss, score

# # 모델 체크포인트 설정 예제 (조정된 모델 저장 경로 사용)
# checkpoint_callback = ModelCheckpoint(
#     dirpath=model_save_dir,
#     filename='best_model-{epoch:02d}-{validation_loss:.4f}',
#     save_top_k=1,
#     verbose=True,
#     monitor='validation_loss',  # 일관된 변수 이름 사용
#     mode='min'
# )

# # EarlyStopping에서 모니터링하는 변수 이름을 일치시킴
# early_stopping_callback = EarlyStopping(
#     monitor='validation_loss',
#     mode='min',
#     patience=3
# )

# trainer = Trainer(
#     max_epochs=CFG['EPOCHS'],
#     callbacks=[checkpoint_callback, early_stopping_callback],
#     gradient_clip_val=1.0,  # 그래디언트 클리핑 값 설정
#     gpus=1,  # CUDA 사용을 명시적으로 지정
#     precision=16  # 16-bit mixed precision 사용으로 메모리 절약 및 속도 향상
# )

# # 학습 및 검증 루프
# generator, discriminator, optimizer_G, optimizer_D = initialize_models_and_optimizers()

# for epoch in range(1, CFG['EPOCHS'] + 1):
#     generator.train()  # Generator 학습 모드
#     for i, batch in enumerate(train_dataloader):
#         loss_G, loss_D = train_step(generator, discriminator, optimizer_G, optimizer_D, batch)
#         # Batch 로그 출력
#         print(f"[Epoch {epoch}] [Batch {i}] [G loss: {loss_G}] [D loss: {loss_D}]")

#     # Validation 단계
#     validation_loss, score = validate(generator, validation_dataloader)

#     best_generator_path = os.path.join(model_save_dir, "best_generator.pth")
#     best_discriminator_path = os.path.join(model_save_dir, "best_discriminator.pth")
        
#     # 모델 저장 조건
#     if score > best_score:
#         best_score = score
#         torch.save(generator.state_dict(), best_generator_path)
#         torch.save(discriminator.state_dict(), best_discriminator_path)

#     # Validation 완료 후 Epoch 로그 출력
#     print(f"Epoch {epoch}: Validation loss: {validation_loss:.4f}, Score: {score:.4f}, Best Score: {best_score:.4f}")


In [ ]:
import cv2
from PIL import Image
import numpy as np
import os

# 저장할 디렉토리 설정
model_save_dir = "./saved_models"
submission_dir = "./submission"
os.makedirs(submission_dir, exist_ok=True)

# 이미지 로드 및 전처리
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image)
    image = image.unsqueeze(0)  # 배치 차원을 추가합니다.
    return image

# 모델 경로 설정
generator_path = os.path.join("/kaggle/input/best-model01/best_generator (1).pth")

# 모델 로드 및 설정
# model = UNetPP(in_channels=3, out_channels=3).to(device)  # UNetPP로 설정
UNetPP.load_state_dict(torch.load(generator_path))
UNetPP.eval()

# 파일 리스트 불러오기
test_images = sorted(os.listdir(test_dir))

# 모든 테스트 이미지에 대해 추론 수행
for image_name in test_images:
    test_image_path = os.path.join(test_dir, image_name)

    # 손상된 테스트 이미지 로드 및 전처리
    test_image = load_image(test_image_path).to(device)

    with torch.no_grad():
        # 모델로 예측
        pred_image = UNetPP(test_image)
        pred_image = pred_image.cpu().squeeze(0)  # 배치 차원 제거
        pred_image = pred_image * 0.5 + 0.5  # 역정규화
        pred_image = pred_image.numpy().transpose(1, 2, 0)  # HWC로 변경
        pred_image = (pred_image * 255).astype('uint8')  # 0-255 범위로 변환
        
        # 예측된 이미지를 실제 이미지와 같은 512x512로 리사이즈
        pred_image_resized = cv2.resize(pred_image, (512, 512), interpolation=cv2.INTER_LINEAR)

    # 결과 이미지 저장
    output_path = os.path.join(submission_dir, image_name)
    cv2.imwrite(output_path, cv2.cvtColor(pred_image_resized, cv2.COLOR_RGB2BGR))    
    
print(f"Saved all images")


In [ ]:
# 저장된 결과 이미지를 ZIP 파일로 압축
zip_filename = "submission1.zip"
with zipfile.ZipFile(zip_filename, 'w') as submission_zip:
    for image_name in test_images:
        image_path = os.path.join(submission_dir, image_name)
        submission_zip.write(image_path, arcname=image_name)

print(f"All images saved in {zip_filename}")

In [ ]:
!tree